# RAG用のファイルデータをベクトルストアに保存するためのnotebook

In [ ]:
from opensearchpy import OpenSearch # pip install opensearch-py
import boto3
import json
import os
from tqdm import tqdm
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

## bedrock接続テスト

In [2]:
# bedrockクライアント
br_client = boto3.client("bedrock-runtime", region_name="us-east-1")

# ベクトルサイズ指定（256/512/1024）
dimensions = 256

# 日本語文章をエンべディングするためのリクエスト
request_body = json.dumps(
    {
        "inputText": "これは日本語の文章です。",
        "dimensions": dimensions,
        "normalize": True
    }
)

# LLMモデル指定＆実行
response = br_client.invoke_model(
    modelId="amazon.titan-embed-text-v2:0", # v2は、次元数を設定できる,
    contentType="application/json",
    accept="application/json",
    body=request_body
)

# 結果確認
response_body = json.loads(response["body"].read())
embedding = response_body.get("embedding")
print(len(embedding))
print(embedding)

256
[-0.04554624482989311, 0.08097110688686371, 0.04225679486989975, 0.032261922955513, 0.13866302371025085, -0.04225679486989975, -0.0268216785043478, 0.023785261437296867, 0.0662950947880745, -0.07692255079746246, -0.0662950947880745, 0.03491878882050514, -0.05237818509340286, -0.05440245941281319, -0.1791485697031021, -0.055920667946338654, -0.01771242916584015, 0.025050435215234756, 0.12348093092441559, 0.04377500340342522, 0.1336023211479187, 0.018598049879074097, -0.04073858633637428, 0.008729697205126286, -0.051113009452819824, -0.09412890672683716, 0.016004445031285286, -0.07945289462804794, 0.05237818509340286, -0.04428107291460037, 0.0049658059142529964, 0.011955889873206615, -0.09109248965978622, -0.041750725358724594, 0.09412890672683716, 0.12348093092441559, 0.007970592938363552, 0.02125491574406624, 0.022014020010828972, 0.02745426446199417, -0.04326893389225006, 0.046305350959300995, -0.0037955206353217363, 0.09210462868213654, 0.06680116057395935, -0.05516156554222107, 

## ベクトルストアへの登録
* 対象ファイル：data/jpbank_economy_report.pdf

### 1. インデックス作成

In [ ]:
# 環境変数読み込み
load_dotenv()

# hostは、【ドメインエンドポイント (IPv4)】を指定する。（awsの対象ドメインのコンソールから確認できる）
host = os.environ["OPENSEARCH_HOST"]

# credentials = boto3.Session().get_credentials()
# auth = AWSV4SignerAuth(credentials, region)
# きめ細かなアクセスコントロールでマスターユーザーを指定した場合は、authの値はドメイン作成時に設定したマスターユーザのユーザ名とパスワード
auth = (os.environ["OPENSEARCH_USER"], os.environ["OPENSEARCH_PASS"])

os_client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
    # connection_class = RequestsHttpConnection
)

In [30]:
# インデックス名設定
index_name = "jp-bank-index"

# インデックス存在確認
os_client.indices.exists(index=index_name)

False

In [ ]:
# os_client.indices.delete(index_name)

{'acknowledged': True}

In [31]:
### コサイン類似度の設定
setting_body ={
    "index.knn": True,
    "index.knn.space_type": "cosinesimil",
    "analysis": {
      "tokenizer": {
        "kuromoji_tokenizer": {
          "type": "kuromoji_tokenizer",
          "mode": "search"
        }
      },
      "analyzer": {
        "my_ja_analyzer": {
          "type": "custom",
          "tokenizer": "kuromoji_tokenizer",
          "filter": [
            "kuromoji_baseform",
            "kuromoji_part_of_speech",
            "cjk_width",
            "ja_stop",
            "kuromoji_stemmer",
            "lowercase"
          ]
        }
      }
    }
}


# インデックス作成用関数
def create_index(index_name, index_body):
    # Create an index with non-default settings.
    # https://docs.aws.amazon.com/opensearch-service/latest/developerguide/sizing-domains.html#bp-sharding
    response = os_client.indices.create(index_name, body=index_body)
    print('\nCreating index:')
    print(response)


# ベクトルサイズ指定（256/512/1024）
dimensions = 256


# マッピングの作成
mapping_body = {
    "properties": {
      "content": {
        "type": "text",
        "analyzer": "my_ja_analyzer"
      },
      "vector_content": {
        "type": "knn_vector",
        "dimension": dimensions,
      }
    }
  }


### インデックス作成
index_body = {}
index_body['settings'] = setting_body
index_body['mappings'] = mapping_body

response = create_index(index_name, index_body)
print()


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'jp-bank-index'}



In [32]:
# 作成したインデックスの確認
os_client.indices.get(index=index_name)

{'jp-bank-index': {'aliases': {},
  'mappings': {'properties': {'content': {'type': 'text',
     'analyzer': 'my_ja_analyzer'},
    'vector_content': {'type': 'knn_vector', 'dimension': 256}}},
  'settings': {'index': {'replication': {'type': 'DOCUMENT'},
    'number_of_shards': '5',
    'provided_name': 'jp-bank-index',
    'knn.space_type': 'cosinesimil',
    'knn': 'true',
    'creation_date': '1732855542397',
    'analysis': {'analyzer': {'my_ja_analyzer': {'filter': ['kuromoji_baseform',
        'kuromoji_part_of_speech',
        'cjk_width',
        'ja_stop',
        'kuromoji_stemmer',
        'lowercase'],
       'type': 'custom',
       'tokenizer': 'kuromoji_tokenizer'}},
     'tokenizer': {'kuromoji_tokenizer': {'mode': 'search',
       'type': 'kuromoji_tokenizer'}}},
    'number_of_replicas': '1',
    'uuid': 'j3EzmQ4TR8aAcZuVAEZuYg',
    'version': {'created': '136387827'}}}}}

### 2. Analyzer機能確認
* 今回はベクトル検索のみなので、Analyzerは使わないが、一応設定したので確認

In [33]:
# アナライザーにかけたいテキスト
text_to_analyze = "原作は、本作で脚本を務める和田竜が手掛けたオリジナルの脚本をもとに自身が書き下ろした小説。2008年に第139回直木賞にノミネートされ、翌年、本屋大賞2位を受賞し、歴史小説ファンはもちろんのこと、若年層から女性層といった幅広い支持を集めている。また本作は、8月下旬から9月初頭にかけてカナダにて開かれる「第36回モントリオール世界映画祭」ワールド・グレイツ部門に特別招待されることが決定している。"

# `_analyze` APIを使ってアナライザーをテスト
analyze_response = os_client.indices.analyze(index=index_name, body={
    "analyzer": "my_ja_analyzer",  # 設定したカスタムアナライザー
    "text": text_to_analyze
})

# 分解されたトークンを表示
for token in analyze_response['tokens']:
    print(f"Token: {token['token']}, Position: {token['position']}")


Token: 原作, Position: 0
Token: 本, Position: 2
Token: 作, Position: 3
Token: 脚本, Position: 5
Token: 務める, Position: 7
Token: 和田, Position: 8
Token: 竜, Position: 9
Token: 手掛ける, Position: 11
Token: オリジナル, Position: 13
Token: 脚本, Position: 15
Token: もと, Position: 17
Token: 自身, Position: 19
Token: 書き下ろす, Position: 21
Token: 小説, Position: 23
Token: 2008, Position: 24
Token: 年, Position: 25
Token: 第, Position: 27
Token: 139, Position: 28
Token: 回, Position: 29
Token: 直木, Position: 30
Token: 直木賞, Position: 30
Token: 賞, Position: 31
Token: ノミネート, Position: 33
Token: 翌年, Position: 36
Token: 本屋, Position: 37
Token: 大賞, Position: 38
Token: 2, Position: 39
Token: 位, Position: 40
Token: 受賞, Position: 42
Token: 歴史, Position: 44
Token: 小説, Position: 45
Token: ファン, Position: 46
Token: もちろん, Position: 48
Token: 若年, Position: 51
Token: 層, Position: 52
Token: 女性, Position: 54
Token: 層, Position: 55
Token: 幅広い, Position: 57
Token: 支持, Position: 58
Token: 集める, Position: 60
Token: 本, Position: 64
Token: 作, Posi

### 3. RAG用ファイル読み込みとチャンキング

In [34]:
# 日銀資料(pdf)の読み込みと文章のチャンク化

# pdfのload
file_path = "./data/jpbank_economy_report.pdf"
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [35]:
print(len(pages))
pages[5].page_content

43


' \n4 \nこうした見通しを前回の展望レポートにおける見通しと比較すると、2025\n年度が、このところの原油等の資源価格下落の影響などから幾分下振れている。 \n消費者物価（除く生鮮食品）の見通しは、原油価格や政府による施策に関す\nる前提にも依存する。原油価格については、先物市場の動向などを参考に、見\n通し期間終盤にかけて緩やかに低下していく前提としている。政府によるガソ\nリン ・電気 ・ ガス代の負担緩和策は、昨年度までの消費者物価 （除く生鮮食品）\nの前年比を押し下げる方向に作用してきた。2024 年度と 2025 年度について\nは、負担緩和策の段階的な縮小・終了が、前年比を押し上げる方向に作用する\nとみられる。エネルギー価格の変動の直接的な影響を受けない消費者物価 （除\nく生鮮食品・エネルギー）の前年比は、既往の輸入物価の上昇を起点とする価\n格転嫁の影響は徐々に減衰するものの、賃金上昇等を受けたサービス価格等の\n緩やかな上昇が続くことから、２％程度で推移するとみられる。 \n物価の基調を規定する主たる要因について点検すると、労働や設備の稼働状\n況を表すマクロ的な需給ギャップは、振れを伴いつつも、改善傾向をたどって\nいる。先行きの需給ギャップは、上記の経済の見通しのもとで、見通し期間終\n盤にかけては、プラス幅を緩やかに拡大していくと予想される。この間、女性\nや高齢者による労働参加の増加ペースの鈍化もあって、 労働需給はマクロ的な\n需給ギャップ以上に引き締まり、賃金の上昇圧力は強まっていくと考えられる。\nこのことは、コスト面では人件費の上昇圧力をもたらすとともに、家計の購買\n力の増加に寄与するとみられる。 \n次に、中長期的な予想物価上昇率をみると、緩やかに上昇している。短観に\nおける企業の物価全般の見通しは、 緩やかに上昇している。適合的予想形成の\n強いわが国において、これまでの物価上昇率の高まりは、 家計や企業の中長期\n的な予想物価上昇率の上昇をもたらしてきている。企業の賃金・価格設定行動\nには、 従来よりも積極的な動きがみられており、名目賃金ははっきりと増加し\nている。また、 賃金の上昇を販売価格に反映する動きも引き続き強まってきて\nいる。先行きについては、需給ギャップの改善が続き、企業の賃金・価格設定\n行

In [ ]:
# チャンク化
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "。", "、", "\n", "！", "？", "；", "：", "「", "」", "『", "』", "（", "）", "・", "…", "　"],
)
all_splits = text_splitter.split_documents(pages)

len(all_splits)

72

In [36]:
i = 0
for chunk in all_splits:
    print(chunk.page_content)
    print("-----------------")
    i += 1
    if i > 4:
        break

経済・物価情勢の展望 
2024 年 10 月 
公表時間 
11 月 1 日（金）14 時 00 分
-----------------
本稿の内容について、商用目的で転載・複製を行う場合（引用は含まれません）
は、予め日本銀行政策委員会室までご相談ください。 
引用・転載・複製を行う場合は、出所を明記してください。
-----------------
1 
 
 
 
経済・物価情勢の展望（2024 年10月） 
【基本的見解】  1 
＜概要＞ 
                              
1 本基本的見解は、10 月 30、31 日開催の政策委員会・金融政策決定会合で決定されたものであ
る。 
 先行きのわが国経済を展望すると、海外経済が緩やかな成長を続けるもとで、緩和的な
金融環境などを背景に、所得から支出への前向きの循環メカニズムが徐々に強まること
から、潜在成長率を上回る成長を続けると考えられる。 
 物価の先行きを展望すると、消費者物価（除く生鮮食品）の前年比は、2024年度に２％
台半ばとなったあと、2025年度および2026年度は、概ね２％程度で推移すると予想さ
れる。既往の輸入物価上昇を起点とする価格転嫁の影響が減衰する一方、消費者物価の基
調的な上昇率は、マクロ的な需給ギャップの改善に加え、賃金と物価の好循環が引き続き
強まり中長期的な予想物価上昇率が上昇していくことから、徐々に高まっていくと予想さ
れ、 見通し期間後半には 「物価安定の目標」 と概ね整合的な水準で推移すると考えられる。
なお、2025年度にかけては、消費者物価（除く生鮮食品）の前年比に対して、政府によ
る施策の反動が押し上げ方向で、 このところの原油等の資源価格下落の影響などが押し下
げ方向で、それぞれ作用すると見込まれる。 
 前回の見通しと比べると、成長率については概ね不変である。消費者物価（除く生鮮食
品） の前年比については、2025年度が、このところの原油等の資源価格下落の影響など
から幾分下振れている。 
 リスク要因をみると、海外の経済・物価動向、資源価格の動向、企業の賃金・価格設定
行動など、わが国経済・物価を巡る不確実性は引き続き高い。そのもとで、金融・為替
市場の動向やそのわが国経済・物価への影響を、十分注視する必要がある。とく

### 4. embeddingとvector storeへの保存

In [38]:
# embedding用関数
def embedding_func(input_text, br_client, dimensions):
    
    # 日本語の文章をエンベディングするためのリクエスト
    request_body = json.dumps(
        {
            "inputText": input_text,
            "dimensions": dimensions,
            "normalize": True
            })
    ### dimensionsとnormaizeは、v2でのみ指定可能！！！

    # モデルIDやcontentTypeを適切に指定してリクエスト
    response = br_client.invoke_model(
        # modelId="amazon.titan-embed-text-v1",
        modelId="amazon.titan-embed-text-v2:0",
        contentType='application/json',
        accept='application/json',
        body=request_body
    )

    # 結果を表示
    response_body = json.loads(response['body'].read())
    embedding = response_body.get('embedding')

    return embedding


# Opensearchに登録する関数
def index_report(content, os_client, br_clinet, dimensions):
    # embedding
    embed_text = embedding_func(content, br_client, dimensions)
    
    doc = {
        "content": content,
        "vector_content": embed_text
    }

    # indexに登録
    res = os_client.index(index=index_name, body=doc)
    print(f"ファイル'{file_path}'がインデックス化されました。ID: {res['_id']}")

    # return doc

In [39]:
# bedrockクライアントの作成
br_client = boto3.client('bedrock-runtime', region_name='us-east-1')

# インデックスにデータ登録
for content in tqdm(all_splits):
    doc = index_report(content.page_content, os_client, br_client, dimensions)

  1%|▏         | 1/72 [00:02<02:23,  2.02s/it]

ファイル'./data/jpbank_economy_report.pdf'がインデックス化されました。ID: 3vs9dpMBygWmEmTv9DkU


  3%|▎         | 2/72 [00:02<01:24,  1.21s/it]

ファイル'./data/jpbank_economy_report.pdf'がインデックス化されました。ID: 3_s9dpMBygWmEmTv-jk8


  4%|▍         | 3/72 [00:03<01:16,  1.10s/it]

...

ファイル'./data/jpbank_economy_report.pdf'がインデックス化されました。ID: I_s-dpMBygWmEmTvhDoX


 99%|█████████▊| 71/72 [00:38<00:00,  2.08it/s]

ファイル'./data/jpbank_economy_report.pdf'がインデックス化されました。ID: JPs-dpMBygWmEmTvhTre


100%|██████████| 72/72 [00:38<00:00,  1.86it/s]

ファイル'./data/jpbank_economy_report.pdf'がインデックス化されました。ID: Jfs-dpMBygWmEmTvhzq0


### 5. 登録確認

In [40]:
# インデックス名と検索したいドキュメントIDを指定
document_id = "IPs-dpMBygWmEmTvfjqD"  # 自動生成されたドキュメントIDを指定

# 特定のIDでドキュメントを取得
try:
    response = os_client.get(index=index_name, id=document_id)
    print("ドキュメントが見つかりました:")
    print(f"ID: {response['_id']}")
    print(f"Source: {response['_source']}")
except Exception as e:
    print("ドキュメントが見つかりません:", e)

ドキュメントが見つかりました:
ID: IPs-dpMBygWmEmTvfjqD
Source: {'content': '38 \nなくなってきていることが示唆された23。こうし\nた事実を踏まえると、わが国では、正社員の人手\n不足感の強まりが企業のこれまでの賃金設定スタ\nンスに変化をもたらしている可能性もある。 \n労働需給を示す各種指標としては、①職業安定\n業務統計の有効求人倍率、②労働経済動向調査の\n欠員率、③正社員求人数なども挙げられる。これ\nらを比較すると、コロナ以降は、有効求人倍率が\n他の指標対比弱めとなっている（図表 B2-5）。こ\nの背景には、公的求人サービスの利用減少と民間\n求人サービスの利用増加により、近年、職業安定\n業務統計の有効求人数が減少している可能性があ\nる24。労働需給のひっ迫度合いについては、こう\nした点にも留意しつつ、今回開発したヒートマッ\nプなども含め、様々な関連指標をみながら評価し\nていくことが適切と考えられる25。 \n  \n                              \n23  さくらレポート別冊「『1990 年代半ば以降の企業行動\n等に関するアンケート調査』の集計結果について」（2024\n年５月）を参照。 \n24 2024 年度の経済財政白書でも、職業安定業務統計の有\n効求人倍率のみで労働需給の引き締まりの程度を確認す\nることが十分でなくなっている点を指摘している。 \n25 労働経済動向調査の欠員率は、 欠員数／常用労働者数と\nして計算。欠員数は、事業 所で仕事があるにも関わ らず、\nその仕事に従事する人がいない状態を補充するために行\nう求人数。常用労働者は、期間を定めずに雇われている者\n及び 1 か月以上の期間を定めて雇われている者の合計。正\n社員求人数指数は正社員が対象で、①民間求人サイトと、\n②ハローワークインターネットサービスで 募集している\n正社員求人の数（重複している求人は調整） 。 \n図表B2-3：労働需給ヒートマップ・業種別\n（出所）厚生労働省\n（注）1. 労働経済動向調査の労働者過不足判断ＤＩを、雇用形態別・業種別に全期間の\n平均と標準偏差で基準化して作成。①の2007年以前は、常用労働者の値を基に\n作成。\n2. 色の表示

### 6. ベクトル検索テスト

In [46]:
search_text = "2025年の物価状況と景気状況"

# Bedrockクライアントの作成
br_client = boto3.client('bedrock-runtime', region_name='us-east-1')

vector_for_query = embedding_func(search_text, br_client, dimensions)

search_query = {
    "size": 5,
    "query": {
        "knn": {
            "vector_content": {
                "vector": vector_for_query, 
                "k": 3
            }
        }
    }
}

results = os_client.search(index=index_name, body=search_query)
for hit in results["hits"]["hits"]:
    print(hit)

{'_index': 'jp-bank-index', '_id': '5fs-dpMBygWmEmTvCDnc', '_score': 0.57236415, '_source': {'content': '4 \nこうした見通しを前回の展望レポートにおける見通しと比較すると、2025\n年度が、このところの原油等の資源価格下落の影響などから幾分下振れている。 \n消費者物価（除く生鮮食品）の見通しは、原油価格や政府による施策に関す\nる前提にも依存する。原油価格については、先物市場の動向などを参考に、見\n通し期間終盤にかけて緩やかに低下していく前提としている。政府によるガソ\nリン ・電気 ・ ガス代の負担緩和策は、昨年度までの消費者物価 （除く生鮮食品）\nの前年比を押し下げる方向に作用してきた。2024 年度と 2025 年度について\nは、負担緩和策の段階的な縮小・終了が、前年比を押し上げる方向に作用する\nとみられる。エネルギー価格の変動の直接的な影響を受けない消費者物価 （除\nく生鮮食品・エネルギー）の前年比は、既往の輸入物価の上昇を起点とする価\n格転嫁の影響は徐々に減衰するものの、賃金上昇等を受けたサービス価格等の\n緩やかな上昇が続くことから、２％程度で推移するとみられる。 \n物価の基調を規定する主たる要因について点検すると、労働や設備の稼働状\n況を表すマクロ的な需給ギャップは、振れを伴いつつも、改善傾向をたどって\nいる。先行きの需給ギャップは、上記の経済の見通しのもとで、見通し期間終\n盤にかけては、プラス幅を緩やかに拡大していくと予想される。この間、女性\nや高齢者による労働参加の増加ペースの鈍化もあって、 労働需給はマクロ的な\n需給ギャップ以上に引き締まり、賃金の上昇圧力は強まっていくと考えられる。\nこのことは、コスト面では人件費の上昇圧力をもたらすとともに、家計の購買\n力の増加に寄与するとみられる。 \n次に、中長期的な予想物価上昇率をみると、緩やかに上昇している。短観に\nおける企業の物価全般の見通しは、 緩やかに上昇している。適合的予想形成の\n強いわが国において、これまでの物価上昇率の高まりは、 家計や企業の中長期\n的な予想物価上昇率の上昇をもたらしてきている。企業の賃金・価格設定行動\nには、 従来よりも積極的な